# Experiment: Trend-Scanning Reproduction (ETF Daily)

## 目的
- `data/prices.csv` の `Adj Close`（9銘柄）で Trend-Scanning ラベルを再現する。
- `reference` 実装と `numba` 最適化実装の整合性を検証する。
- リーク防止（時点整合）と分布診断を確認する。

## スコープ外
- Fixed Horizon / Triple Barrier 比較
- Meta-labeling 学習
- EURUSD M5 の原論文数値との完全一致


In [1]:
from __future__ import annotations

import os
import sys
import time
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.api as sm
from numba import njit, prange

np.random.seed(42)

DATA_PATH = Path('/Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/data/prices.csv')
OUTPUT_EVIDENCE_PATH = Path('/Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/outputs/20260223/phase_b_evidence.md')
NOTEBOOK_PATH = Path('/Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/trend_scanning_reproduction.ipynb')

SPAN = (5, 100)
VOL_THRESHOLD = 0.05
USE_LOG = True
TICKERS = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']

OUTPUT_EVIDENCE_PATH.parent.mkdir(parents=True, exist_ok=True)
os.environ.setdefault('MPLCONFIGDIR', '/tmp/mplconfig')

print('DATA_PATH exists:', DATA_PATH.exists())
print('OUTPUT_EVIDENCE_PATH:', OUTPUT_EVIDENCE_PATH)
print('Python:', sys.version.split()[0])
print('NumPy:', np.__version__)
print('Pandas:', pd.__version__)
print('Statsmodels:', sm.__version__)


DATA_PATH exists: True
OUTPUT_EVIDENCE_PATH: /Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/outputs/20260223/phase_b_evidence.md
Python: 3.12.12
NumPy: 2.4.2
Pandas: 3.0.0
Statsmodels: 0.14.6


In [2]:
raw = pd.read_csv(DATA_PATH, header=[0, 1], index_col=0)
raw.index = pd.to_datetime(raw.index)

adj = raw['Adj Close'].copy().sort_index()
adj = adj.loc[:, TICKERS]

assert isinstance(adj.index, pd.DatetimeIndex)
assert adj.index.is_monotonic_increasing
assert adj.index.is_unique
assert not adj.isna().any().any()
assert (adj > 0).all().all()

print('adj shape:', adj.shape)
print('date range:', adj.index.min().date(), '->', adj.index.max().date())
print('tickers:', list(adj.columns))
adj.head(3)


adj shape: (3806, 9)
date range: 2011-01-03 -> 2026-02-20
tickers: ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']


Ticker            XLB        XLE        XLF        XLI        XLK        XLP  \
Date                                                                           
2011-01-03  14.224260  20.671648  10.076117  26.505562  10.422182  19.617113   
2011-01-04  14.128747  20.494282  10.063756  26.467825  10.463102  19.563637   
2011-01-05  14.161809  20.575447  10.181208  26.603682  10.524482  19.570326   

Ticker           XLU        XLV        XLY  
Date                                        
2011-01-03  9.450070  24.678146  15.783327  
2011-01-04  9.495085  24.755775  15.695760  
2011-01-05  9.435068  24.810118  15.825027

In [3]:
REQUIRED_COLUMNS = ['t1', 'window', 'slope', 't_value', 'rsquared', 'ret', 'bin']


def _expand_horizons(span: tuple[int, int] | list[int]) -> list[int]:
    if isinstance(span, tuple):
        hrzns = list(range(*span))
    else:
        hrzns = list(span)
    if len(hrzns) == 0:
        raise ValueError('span must contain at least one horizon')
    if min(hrzns) < 2:
        raise ValueError('minimum horizon must be >= 2')
    return hrzns


def validate_series(close: pd.Series) -> None:
    if not isinstance(close, pd.Series):
        raise TypeError('close must be pd.Series')
    if not isinstance(close.index, pd.DatetimeIndex):
        raise TypeError('close.index must be DatetimeIndex')
    if not close.index.is_monotonic_increasing:
        raise ValueError('close.index must be monotonic increasing')
    if not close.index.is_unique:
        raise ValueError('close.index must be unique')
    if close.isna().any():
        raise ValueError('close contains NaN')
    if (close <= 0).any():
        raise ValueError('close must be strictly positive')


def validate_output_schema(df: pd.DataFrame) -> None:
    if list(df.columns) != REQUIRED_COLUMNS:
        raise ValueError(f'unexpected columns: {list(df.columns)}')
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError('output index must be DatetimeIndex')
    if df['bin'].dtype != np.int8:
        raise TypeError('bin dtype must be int8')
    allowed = {-1, 0, 1}
    values = set(df['bin'].dropna().astype(int).tolist())
    if not values.issubset(allowed):
        raise ValueError(f'bin has unexpected labels: {values}')
    finite_cols = ['window', 'slope', 't_value', 'rsquared', 'ret']
    if not np.isfinite(df[finite_cols].to_numpy(dtype=float)).all():
        raise ValueError('non-finite values found in output metrics')


def check_temporal_integrity(df: pd.DataFrame, lookforward: bool) -> None:
    if lookforward:
        if not (df['t1'] >= df.index).all():
            raise ValueError('lookforward=True requires t1 >= t')
    else:
        if not (df['t1'] <= df.index).all():
            raise ValueError('lookforward=False requires t1 <= t')


def compare_frames(df_ref: pd.DataFrame, df_num: pd.DataFrame) -> dict[str, float]:
    if not df_ref.index.equals(df_num.index):
        raise ValueError('index mismatch between reference and numba outputs')

    out = {}
    out['n_rows'] = float(len(df_ref))
    out['schema_match'] = float(list(df_ref.columns) == list(df_num.columns))
    out['index_match'] = float(df_ref.index.equals(df_num.index))
    out['t1_match_rate'] = float((df_ref['t1'] == df_num['t1']).mean())
    out['window_match_rate'] = float((df_ref['window'] == df_num['window']).mean())
    out['bin_match_rate'] = float((df_ref['bin'] == df_num['bin']).mean())

    delta_t = np.abs(df_ref['t_value'].to_numpy() - df_num['t_value'].to_numpy())
    delta_s = np.abs(df_ref['slope'].to_numpy() - df_num['slope'].to_numpy())
    delta_r = np.abs(df_ref['rsquared'].to_numpy() - df_num['rsquared'].to_numpy())
    delta_ret = np.abs(df_ref['ret'].to_numpy() - df_num['ret'].to_numpy())

    out['t_value_mae'] = float(delta_t.mean())
    out['t_value_max_abs_diff'] = float(delta_t.max())
    out['slope_mae'] = float(delta_s.mean())
    out['rsquared_mae'] = float(delta_r.mean())
    out['ret_mae'] = float(delta_ret.mean())
    return out


## Reference Implementation

- `t_val_linr` は `statsmodels.OLS` を使った忠実な単窓 t-value 計算関数。
- ただし全銘柄・全窓で毎回 `OLS.fit()` を回すと計算量が大きいため、
  `trend_scanning_labels_reference` 本体は同じ OLS 定義に基づく閉形式計算を使う。
- 次セルで `statsmodels` と閉形式の t-value 同値性をサンプル検証する。


In [4]:
def t_val_linr(close_window: np.ndarray) -> float:
    y = np.asarray(close_window, dtype=np.float64)
    x = np.ones((y.shape[0], 2), dtype=np.float64)
    x[:, 1] = np.arange(y.shape[0], dtype=np.float64)
    model = sm.OLS(y, x).fit()
    return float(model.tvalues[1])


def _t_val_linr_closed_form(close_window: np.ndarray) -> float:
    y = np.asarray(close_window, dtype=np.float64)
    w = y.shape[0]
    t = np.arange(w, dtype=np.float64)
    mean_t = t.mean()
    var_t = ((t - mean_t) ** 2).sum()
    eps = 1e-12

    sum_y = y.sum()
    sum_y2 = (y ** 2).sum()
    s_ty = (y * t).sum()
    mean_y = sum_y / w

    slope = (s_ty - w * mean_t * mean_y) / var_t
    beta0 = mean_y - slope * mean_t
    sse = sum_y2 - beta0 * sum_y - slope * s_ty
    sigma2 = sse / (w - 2 + eps)
    se_slope = np.sqrt(sigma2 / var_t)
    return float(slope / (se_slope + eps))


def _window_stats_python(y: np.ndarray, window_length: int) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    y = np.asarray(y, dtype=np.float64)
    n = y.shape[0]
    num_windows = n - window_length + 1
    if num_windows <= 0:
        return np.empty(0), np.empty(0), np.empty(0)

    idx = np.arange(n, dtype=np.float64)
    csum = np.concatenate(([0.0], np.cumsum(y)))
    csum2 = np.concatenate(([0.0], np.cumsum(y * y)))
    csum_jy = np.concatenate(([0.0], np.cumsum(idx * y)))

    start = np.arange(num_windows)
    end = start + window_length

    sum_y = csum[end] - csum[start]
    sum_y2 = csum2[end] - csum2[start]
    sum_jy = csum_jy[end] - csum_jy[start]

    t = np.arange(window_length, dtype=np.float64)
    mean_t = t.mean()
    var_t = ((t - mean_t) ** 2).sum()
    eps = 1e-12

    mean_y = sum_y / window_length
    s_ty = sum_jy - start * sum_y

    slope = (s_ty - window_length * mean_t * mean_y) / var_t
    beta0 = mean_y - slope * mean_t
    sse = sum_y2 - beta0 * sum_y - slope * s_ty
    sst = sum_y2 - (sum_y * sum_y) / window_length

    sigma2 = sse / (window_length - 2 + eps)
    se_slope = np.sqrt(np.maximum(sigma2, 0.0) / var_t)
    t_values = slope / (se_slope + eps)

    rsquared = np.where(sst > eps, 1.0 - sse / (sst + eps), 0.0)
    rsquared = np.clip(rsquared, 0.0, 1.0)

    return t_values, slope, rsquared


def trend_scanning_labels_reference(
    close: pd.Series,
    span: tuple[int, int] = (5, 100),
    lookforward: bool = True,
    use_log: bool = True,
    volatility_threshold: float = 0.05,
) -> pd.DataFrame:
    validate_series(close)

    close = close.sort_index().astype(np.float64)
    hrzns = _expand_horizons(span)
    max_hrzn = max(hrzns)

    if lookforward:
        valid_indices = close.index[:-max_hrzn].to_list()
    else:
        valid_indices = close.index[max_hrzn - 1 :].to_list()

    if not valid_indices:
        empty = pd.DataFrame(columns=REQUIRED_COLUMNS, index=pd.DatetimeIndex([], name=close.index.name))
        empty['bin'] = empty['bin'].astype(np.int8)
        return empty

    if use_log:
        y = np.log(close.clip(lower=1e-8).to_numpy(dtype=np.float64))
    else:
        y = close.to_numpy(dtype=np.float64)

    n = len(y)
    volatility = pd.Series(y, index=close.index).rolling(max_hrzn, min_periods=1).std()
    vol_threshold = volatility.expanding().quantile(volatility_threshold).ffill().to_numpy()

    window_stats = np.full((3, n, len(hrzns)), np.nan, dtype=np.float64)

    for k, hrzn in enumerate(hrzns):
        y_window = y if lookforward else y[::-1]
        t_vals, slopes, r_sq = _window_stats_python(y_window, hrzn)

        if not lookforward:
            t_vals = t_vals[::-1]
            slopes = slopes[::-1]
            r_sq = r_sq[::-1]
            start_idx = hrzn - 1
        else:
            start_idx = 0

        m = len(t_vals)
        vol_slice = volatility.iloc[start_idx : start_idx + m].to_numpy()
        mask = vol_slice > vol_threshold[start_idx : start_idx + m]

        window_stats[0, start_idx : start_idx + m, k] = np.where(mask, t_vals, 0.0)
        window_stats[1, start_idx : start_idx + m, k] = np.where(mask, slopes, 0.0)
        window_stats[2, start_idx : start_idx + m, k] = np.where(mask, r_sq, 0.0)

    event_idx = close.index.get_indexer(valid_indices)
    t_block = window_stats[0, event_idx, :]
    s_block = window_stats[1, event_idx, :]
    rsq_block = window_stats[2, event_idx, :]

    best_j = np.nanargmax(np.abs(t_block), axis=1)

    opt_t = t_block[np.arange(len(event_idx)), best_j]
    opt_s = s_block[np.arange(len(event_idx)), best_j]
    opt_r = rsq_block[np.arange(len(event_idx)), best_j]
    opt_w = np.asarray(hrzns, dtype=np.int64)[best_j]

    if lookforward:
        t1_idx = np.clip(event_idx + opt_w - 1, 0, n - 1)
        a, b = event_idx, t1_idx
    else:
        t1_idx = np.clip(event_idx - opt_w + 1, 0, n - 1)
        a, b = t1_idx, event_idx

    t1_arr = close.index[t1_idx]
    close_values = close.to_numpy(dtype=np.float64)
    rets = close_values[b] / close_values[a] - 1.0

    bins = np.where(np.abs(opt_t) > 1e-6, np.sign(opt_t), 0).astype(np.int8)

    out = pd.DataFrame(
        {
            't1': t1_arr,
            'window': opt_w.astype(np.int64),
            'slope': opt_s,
            't_value': opt_t,
            'rsquared': opt_r,
            'ret': rets,
            'bin': bins,
        },
        index=pd.DatetimeIndex(valid_indices, name=close.index.name),
    )

    out = out[REQUIRED_COLUMNS]
    out['window'] = out['window'].astype(np.int32)
    out['bin'] = out['bin'].astype(np.int8)

    validate_output_schema(out)
    check_temporal_integrity(out, lookforward=lookforward)
    return out


## Numba Optimized Implementation

- `_window_stats_numba` は窓ごとの OLS 統計量を JIT コンパイルで計算する。
- `trend_scanning_labels_numba` は reference と同じ I/O 仕様を維持する。


In [5]:
@njit(parallel=True, cache=True)
def _window_stats_numba(y: np.ndarray, window_length: int):
    n = len(y)
    num_windows = n - window_length + 1
    t_values = np.empty(num_windows, dtype=np.float64)
    slopes = np.empty(num_windows, dtype=np.float64)
    rsquared = np.empty(num_windows, dtype=np.float64)

    t = np.arange(window_length, dtype=np.float64)
    mean_t = t.mean()
    var_t = ((t - mean_t) ** 2).sum()
    eps = 1e-12

    for i in prange(num_windows):
        sum_y = 0.0
        sum_y2 = 0.0
        s_ty = 0.0

        for k in range(window_length):
            v = y[i + k]
            sum_y += v
            sum_y2 += v * v
            s_ty += v * k

        mean_y = sum_y / window_length
        slope = (s_ty - window_length * mean_t * mean_y) / var_t
        slopes[i] = slope

        beta0 = mean_y - slope * mean_t
        sse = sum_y2 - beta0 * sum_y - slope * s_ty
        sst = sum_y2 - (sum_y * sum_y) / window_length

        if sst > eps:
            rs = 1.0 - sse / (sst + eps)
            if rs < 0.0:
                rs = 0.0
            elif rs > 1.0:
                rs = 1.0
            rsquared[i] = rs
        else:
            rsquared[i] = 0.0

        sigma2 = sse / (window_length - 2 + eps)
        if sigma2 < 0.0:
            sigma2 = 0.0
        se_slope = np.sqrt(sigma2 / var_t)
        t_values[i] = slope / (se_slope + eps)

    return t_values, slopes, rsquared


def trend_scanning_labels_numba(
    close: pd.Series,
    span: tuple[int, int] = (5, 100),
    lookforward: bool = True,
    use_log: bool = True,
    volatility_threshold: float = 0.05,
) -> pd.DataFrame:
    validate_series(close)

    close = close.sort_index().astype(np.float64)
    hrzns = _expand_horizons(span)
    max_hrzn = max(hrzns)

    if lookforward:
        valid_indices = close.index[:-max_hrzn].to_list()
    else:
        valid_indices = close.index[max_hrzn - 1 :].to_list()

    if not valid_indices:
        empty = pd.DataFrame(columns=REQUIRED_COLUMNS, index=pd.DatetimeIndex([], name=close.index.name))
        empty['bin'] = empty['bin'].astype(np.int8)
        return empty

    if use_log:
        y = np.log(close.clip(lower=1e-8).to_numpy(dtype=np.float64))
    else:
        y = close.to_numpy(dtype=np.float64)

    n = len(y)
    volatility = pd.Series(y, index=close.index).rolling(max_hrzn, min_periods=1).std()
    vol_threshold = volatility.expanding().quantile(volatility_threshold).ffill().to_numpy()

    window_stats = np.full((3, n, len(hrzns)), np.nan, dtype=np.float64)

    for k, hrzn in enumerate(hrzns):
        y_window = y if lookforward else y[::-1]
        t_vals, slopes, r_sq = _window_stats_numba(y_window, hrzn)

        if not lookforward:
            t_vals = t_vals[::-1]
            slopes = slopes[::-1]
            r_sq = r_sq[::-1]
            start_idx = hrzn - 1
        else:
            start_idx = 0

        m = len(t_vals)
        vol_slice = volatility.iloc[start_idx : start_idx + m].to_numpy()
        mask = vol_slice > vol_threshold[start_idx : start_idx + m]

        window_stats[0, start_idx : start_idx + m, k] = np.where(mask, t_vals, 0.0)
        window_stats[1, start_idx : start_idx + m, k] = np.where(mask, slopes, 0.0)
        window_stats[2, start_idx : start_idx + m, k] = np.where(mask, r_sq, 0.0)

    event_idx = close.index.get_indexer(valid_indices)
    t_block = window_stats[0, event_idx, :]
    s_block = window_stats[1, event_idx, :]
    rsq_block = window_stats[2, event_idx, :]

    best_j = np.nanargmax(np.abs(t_block), axis=1)

    opt_t = t_block[np.arange(len(event_idx)), best_j]
    opt_s = s_block[np.arange(len(event_idx)), best_j]
    opt_r = rsq_block[np.arange(len(event_idx)), best_j]
    opt_w = np.asarray(hrzns, dtype=np.int64)[best_j]

    if lookforward:
        t1_idx = np.clip(event_idx + opt_w - 1, 0, n - 1)
        a, b = event_idx, t1_idx
    else:
        t1_idx = np.clip(event_idx - opt_w + 1, 0, n - 1)
        a, b = t1_idx, event_idx

    t1_arr = close.index[t1_idx]
    close_values = close.to_numpy(dtype=np.float64)
    rets = close_values[b] / close_values[a] - 1.0

    bins = np.where(np.abs(opt_t) > 1e-6, np.sign(opt_t), 0).astype(np.int8)

    out = pd.DataFrame(
        {
            't1': t1_arr,
            'window': opt_w.astype(np.int64),
            'slope': opt_s,
            't_value': opt_t,
            'rsquared': opt_r,
            'ret': rets,
            'bin': bins,
        },
        index=pd.DatetimeIndex(valid_indices, name=close.index.name),
    )

    out = out[REQUIRED_COLUMNS]
    out['window'] = out['window'].astype(np.int32)
    out['bin'] = out['bin'].astype(np.int8)

    validate_output_schema(out)
    check_temporal_integrity(out, lookforward=lookforward)
    return out


In [6]:
# 1) statsmodels t-value と閉形式 t-value の同値性チェック
series_xlk = adj['XLK'].copy()
validate_series(series_xlk)

rng = np.random.default_rng(42)
window = 30
starts = rng.integers(0, len(series_xlk) - window, size=20)

sm_vals = []
cf_vals = []
for s in starts:
    w = series_xlk.iloc[s : s + window].to_numpy(dtype=np.float64)
    sm_vals.append(t_val_linr(w))
    cf_vals.append(_t_val_linr_closed_form(w))

sm_vals = np.array(sm_vals)
cf_vals = np.array(cf_vals)
print('statsmodels vs closed-form max abs diff:', float(np.max(np.abs(sm_vals - cf_vals))))

# 2) XLK で reference / numba のサニティ比較
_ = trend_scanning_labels_numba(series_xlk.iloc[:400], span=SPAN, lookforward=True, use_log=USE_LOG, volatility_threshold=VOL_THRESHOLD)

st = time.perf_counter()
xlk_ref = trend_scanning_labels_reference(series_xlk, span=SPAN, lookforward=True, use_log=USE_LOG, volatility_threshold=VOL_THRESHOLD)
ref_time = time.perf_counter() - st

st = time.perf_counter()
xlk_num = trend_scanning_labels_numba(series_xlk, span=SPAN, lookforward=True, use_log=USE_LOG, volatility_threshold=VOL_THRESHOLD)
num_time = time.perf_counter() - st

validate_output_schema(xlk_ref)
validate_output_schema(xlk_num)
check_temporal_integrity(xlk_ref, lookforward=True)
check_temporal_integrity(xlk_num, lookforward=True)

xlk_metrics = compare_frames(xlk_ref, xlk_num)
xlk_metrics['reference_time_sec'] = ref_time
xlk_metrics['numba_time_sec'] = num_time
xlk_metrics['speedup_x'] = ref_time / num_time if num_time > 0 else np.nan

pd.DataFrame([xlk_metrics]).T.rename(columns={0: 'XLK'})


statsmodels vs closed-form max abs diff: 3.382979230082128e-09


XLK
n_rows                3.707000e+03
schema_match          1.000000e+00
index_match           1.000000e+00
t1_match_rate         1.000000e+00
window_match_rate     1.000000e+00
bin_match_rate        1.000000e+00
t_value_mae           4.741321e-07
t_value_max_abs_diff  3.873310e-04
slope_mae             3.790178e-12
rsquared_mae          1.066811e-09
ret_mae               0.000000e+00
reference_time_sec    1.725075e-02
numba_time_sec        2.589025e-02
speedup_x             6.663030e-01

## 完全忠実 Reference（`statsmodels.OLS.fit` 全窓）検証

- 本セルでは、各イベント×各ホライズンで `statsmodels.OLS.fit()` を明示的に実行する忠実実装を追加する。
- 計算コストが高いため、検証は代表銘柄のサブセットで行い、
  既存 `reference`（閉形式）および `numba` 実装と整合性を比較する。


In [7]:
def trend_scanning_labels_reference_smfit(
    close: pd.Series,
    span: tuple[int, int] = (5, 100),
    lookforward: bool = True,
    use_log: bool = True,
    volatility_threshold: float = 0.05,
) -> pd.DataFrame:
    validate_series(close)

    close = close.sort_index().astype(np.float64)
    hrzns = _expand_horizons(span)
    max_hrzn = max(hrzns)

    if lookforward:
        valid_pos = np.arange(0, len(close) - max_hrzn)
    else:
        valid_pos = np.arange(max_hrzn - 1, len(close))

    if len(valid_pos) == 0:
        empty = pd.DataFrame(columns=REQUIRED_COLUMNS, index=pd.DatetimeIndex([], name=close.index.name))
        empty['bin'] = empty['bin'].astype(np.int8)
        return empty

    if use_log:
        y = np.log(close.clip(lower=1e-8).to_numpy(dtype=np.float64))
    else:
        y = close.to_numpy(dtype=np.float64)

    n = len(y)
    volatility = pd.Series(y, index=close.index).rolling(max_hrzn, min_periods=1).std()
    vol_threshold = volatility.expanding().quantile(volatility_threshold).ffill().to_numpy()

    rows = []
    idx_list = []

    for i in valid_pos:
        best_abs_t = -1.0
        best = None

        for h in hrzns:
            if lookforward:
                st = i
                en = i + h
                if en > n:
                    continue
                t1_pos = en - 1
            else:
                st = i - h + 1
                en = i + 1
                if st < 0:
                    continue
                t1_pos = st

            vol_ok = volatility.iloc[i] > vol_threshold[i]
            if not vol_ok:
                t_val = 0.0
                slope = 0.0
                rsq = 0.0
            else:
                y_win = y[st:en]
                x = np.ones((len(y_win), 2), dtype=np.float64)
                x[:, 1] = np.arange(len(y_win), dtype=np.float64)
                fit = sm.OLS(y_win, x).fit()
                t_val = float(fit.tvalues[1])
                slope = float(fit.params[1])
                rsq_raw = float(fit.rsquared) if np.isfinite(fit.rsquared) else 0.0
                rsq = min(1.0, max(0.0, rsq_raw))

            abs_t = abs(t_val)
            if abs_t > best_abs_t:
                best_abs_t = abs_t
                best = (t1_pos, h, slope, t_val, rsq)

        if best is None:
            continue

        t1_pos, h_opt, slope_opt, t_opt, rsq_opt = best
        if lookforward:
            a_pos, b_pos = i, t1_pos
        else:
            a_pos, b_pos = t1_pos, i

        ret = close.iloc[b_pos] / close.iloc[a_pos] - 1.0
        bin_v = int(np.sign(t_opt)) if abs(t_opt) > 1e-6 else 0

        rows.append(
            {
                't1': close.index[t1_pos],
                'window': int(h_opt),
                'slope': float(slope_opt),
                't_value': float(t_opt),
                'rsquared': float(rsq_opt),
                'ret': float(ret),
                'bin': np.int8(bin_v),
            }
        )
        idx_list.append(close.index[i])

    out = pd.DataFrame(rows, index=pd.DatetimeIndex(idx_list, name=close.index.name))
    out = out[REQUIRED_COLUMNS]
    out['window'] = out['window'].astype(np.int32)
    out['bin'] = out['bin'].astype(np.int8)

    validate_output_schema(out)
    check_temporal_integrity(out, lookforward=lookforward)
    return out


# 忠実実装の検証（計算コスト抑制のためサブセット）
smfit_ticker = 'XLK'
smfit_span = (5, 20)
smfit_series = adj[smfit_ticker].iloc[:700].copy()

st = time.perf_counter()
smfit_df = trend_scanning_labels_reference_smfit(
    smfit_series,
    span=smfit_span,
    lookforward=True,
    use_log=USE_LOG,
    volatility_threshold=VOL_THRESHOLD,
)
smfit_elapsed = time.perf_counter() - st

st = time.perf_counter()
fast_ref_df = trend_scanning_labels_reference(
    smfit_series,
    span=smfit_span,
    lookforward=True,
    use_log=USE_LOG,
    volatility_threshold=VOL_THRESHOLD,
)
fast_ref_elapsed = time.perf_counter() - st

st = time.perf_counter()
numba_df = trend_scanning_labels_numba(
    smfit_series,
    span=smfit_span,
    lookforward=True,
    use_log=USE_LOG,
    volatility_threshold=VOL_THRESHOLD,
)
numba_elapsed = time.perf_counter() - st

smfit_vs_fast = compare_frames(smfit_df, fast_ref_df)
smfit_vs_numba = compare_frames(smfit_df, numba_df)

smfit_compare_df = pd.DataFrame([
    {
        'pair': 'smfit_vs_fast_ref',
        **smfit_vs_fast,
        'lhs_time_sec': smfit_elapsed,
        'rhs_time_sec': fast_ref_elapsed,
    },
    {
        'pair': 'smfit_vs_numba',
        **smfit_vs_numba,
        'lhs_time_sec': smfit_elapsed,
        'rhs_time_sec': numba_elapsed,
    },
]).set_index('pair')

smfit_compare_df


n_rows  schema_match  index_match  t1_match_rate  \
pair                                                                  
smfit_vs_fast_ref   681.0           1.0          1.0            1.0   
smfit_vs_numba      681.0           1.0          1.0            1.0   

                   window_match_rate  bin_match_rate   t_value_mae  \
pair                                                                 
smfit_vs_fast_ref                1.0             1.0  3.599287e-08   
smfit_vs_numba                   1.0             1.0  1.671890e-08   

                   t_value_max_abs_diff     slope_mae  rsquared_mae  ret_mae  \
pair                                                                           
smfit_vs_fast_ref          2.512811e-06  8.001509e-13  8.002002e-10      0.0   
smfit_vs_numba             1.240881e-07  1.488567e-16  3.380461e-10      0.0   

                   lhs_time_sec  rhs_time_sec  
pair                                           
smfit_vs_fast_ref      0.531943      0.002303  
smfit_vs_numba         0.531943      0.004447

In [8]:
results = []
numba_warmed_up = False

for ticker in TICKERS:
    s = adj[ticker].copy()
    validate_series(s)

    if not numba_warmed_up:
        _ = trend_scanning_labels_numba(s.iloc[:400], span=SPAN, lookforward=True, use_log=USE_LOG, volatility_threshold=VOL_THRESHOLD)
        numba_warmed_up = True

    st = time.perf_counter()
    df_ref = trend_scanning_labels_reference(s, span=SPAN, lookforward=True, use_log=USE_LOG, volatility_threshold=VOL_THRESHOLD)
    ref_elapsed = time.perf_counter() - st

    st = time.perf_counter()
    df_num = trend_scanning_labels_numba(s, span=SPAN, lookforward=True, use_log=USE_LOG, volatility_threshold=VOL_THRESHOLD)
    num_elapsed = time.perf_counter() - st

    m = compare_frames(df_ref, df_num)
    m['ticker'] = ticker
    m['reference_time_sec'] = ref_elapsed
    m['numba_time_sec'] = num_elapsed
    m['speedup_x'] = ref_elapsed / num_elapsed if num_elapsed > 0 else np.nan
    results.append(m)

all_metrics_df = pd.DataFrame(results).set_index('ticker').sort_index()
all_metrics_df


n_rows  schema_match  index_match  t1_match_rate  window_match_rate  \
ticker                                                                        
XLB     3707.0           1.0          1.0            1.0                1.0   
XLE     3707.0           1.0          1.0            1.0                1.0   
XLF     3707.0           1.0          1.0            1.0                1.0   
XLI     3707.0           1.0          1.0            1.0                1.0   
XLK     3707.0           1.0          1.0            1.0                1.0   
XLP     3707.0           1.0          1.0            1.0                1.0   
XLU     3707.0           1.0          1.0            1.0                1.0   
XLV     3707.0           1.0          1.0            1.0                1.0   
XLY     3707.0           1.0          1.0            1.0                1.0   

        bin_match_rate   t_value_mae  t_value_max_abs_diff     slope_mae  \
ticker                                                                     
XLB                1.0  6.151288e-07              0.000554  4.944006e-12   
XLE                1.0  3.979636e-07              0.000206  5.811532e-12   
XLF                1.0  7.847778e-07              0.001563  4.042914e-12   
XLI                1.0  1.115044e-06              0.001786  4.708104e-12   
XLK                1.0  4.741321e-07              0.000387  3.790178e-12   
XLP                1.0  7.792423e-07              0.000875  4.220058e-12   
XLU                1.0  2.045122e-06              0.004940  4.757084e-12   
XLV                1.0  1.035334e-06              0.000749  5.425511e-12   
XLY                1.0  6.279230e-07              0.000497  3.921129e-12   

        rsquared_mae  ret_mae  reference_time_sec  numba_time_sec  speedup_x  
ticker                                                                        
XLB     1.409810e-09      0.0            0.015196        0.025634   0.592806  
XLE     1.269387e-09      0.0            0.014510        0.025900   0.560255  
XLF     1.374376e-09      0.0            0.014653        0.026456   0.553845  
XLI     1.739524e-09      0.0            0.014108        0.026193   0.538596  
XLK     1.066811e-09      0.0            0.014108        0.026700   0.528399  
XLP     2.182380e-09      0.0            0.014015        0.025719   0.544918  
XLU     1.440112e-09      0.0            0.013893        0.025944   0.535511  
XLV     1.918280e-09      0.0            0.014025        0.025269   0.555030  
XLY     1.117250e-09      0.0            0.046723        0.026796   1.743686

## 分布診断

以下は `numba` 実装出力を使った分布診断。


In [9]:
label_outputs = {}
for ticker in TICKERS:
    s = adj[ticker].copy()
    label_outputs[ticker] = trend_scanning_labels_numba(
        s,
        span=SPAN,
        lookforward=True,
        use_log=USE_LOG,
        volatility_threshold=VOL_THRESHOLD,
    )

bin_dist = pd.DataFrame({
    t: label_outputs[t]['bin'].value_counts().reindex([-1, 0, 1], fill_value=0)
    for t in TICKERS
}).T
bin_dist.columns = ['bin_-1', 'bin_0', 'bin_+1']

window_top = {}
for t in TICKERS:
    vc = label_outputs[t]['window'].value_counts().sort_index()
    window_top[t] = vc.nlargest(10)

window_top_df = pd.DataFrame(window_top).fillna(0).astype(int).T

q_levels = [0.0, 0.25, 0.5, 0.75, 1.0]
tval_quantiles = pd.DataFrame({
    t: label_outputs[t]['t_value'].quantile(q_levels).to_numpy()
    for t in TICKERS
}, index=[f'q{int(q*100):02d}' for q in q_levels]).T

dist_flags = pd.DataFrame(index=TICKERS)
dist_flags['single_bin_flag'] = [int(label_outputs[t]['bin'].nunique() == 1) for t in TICKERS]
dist_flags['window_concentration_flag'] = [int(label_outputs[t]['window'].value_counts(normalize=True).iloc[0] >= 0.95) for t in TICKERS]
dist_flags['non_finite_flag'] = [int(~np.isfinite(label_outputs[t][['window', 'slope', 't_value', 'rsquared', 'ret']].to_numpy(dtype=float)).all()) for t in TICKERS]

print('bin distribution')
display(bin_dist)
print('top-10 window counts per ticker')
display(window_top_df)
print('t_value quantiles')
display(tval_quantiles)
print('distribution anomaly flags (0=normal, 1=alert)')
display(dist_flags)


bin distribution


bin_-1  bin_0  bin_+1
XLB    1189    189    2329
XLE    1426    121    2160
XLF     968     30    2709
XLI     970    186    2551
XLK     757     62    2888
XLP     956     22    2729
XLU     882      6    2819
XLV     900    102    2705
XLY     829    136    2742

top-10 window counts per ticker


window   5   6   16  19  20  24  25  26  27  28  ...  85  87  89  90  93  95  \
XLB     239  36   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
XLE     164   0   0  36  38   0   0   0   0   0  ...   0   0  36  37  36  37   
XLF      65   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0  35   
XLI     223   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
XLK      90   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
XLP      55   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0  38   
XLU      51   0  40   0  40   0   0   0   0   0  ...   0   0   0   0   0   0   
XLV     146   0   0   0   0   0   0   0   0   0  ...  37  34   0   0   0   0   
XLY     172   0   0   0   0  38  39  42  39  38  ...   0   0   0   0   0   0   

window  96  97  98    99  
XLB      0  36  47   728  
XLE      0  42  47   879  
XLF     36  39  47  1070  
XLI      0  34  42  1054  
XLK     38  42  55  1111  
XLP     42  45  53   911  
XLU      0   0  51   786  
XLV      0  39  55  1012  
XLY      0   0  42   863  

[9 rows x 42 columns]

t_value quantiles


q00        q25        q50        q75       q100
XLB -35.651384  -8.744892  10.068987  17.272163  48.524180
XLE -41.129445 -10.750814   8.938909  15.512630  41.426353
XLF -44.376414  -5.930105  12.230169  18.889787  56.538105
XLI -39.726907  -6.014777  12.432454  18.809065  46.416381
XLK -25.548642   7.563859  15.283724  22.843555  69.228779
XLP -30.354925  -5.776561  13.038519  21.444284  77.504553
XLU -50.248346   6.041667  12.908046  19.715623  57.122492
XLV -40.774523   0.000000  13.603284  20.613825  52.518573
XLY -56.619480   0.000000  14.039903  21.119209  51.872553

distribution anomaly flags (0=normal, 1=alert)


single_bin_flag  window_concentration_flag  non_finite_flag
XLB                0                          0                0
XLE                0                          0                0
XLF                0                          0                0
XLI                0                          0                0
XLK                0                          0                0
XLP                0                          0                0
XLU                0                          0                0
XLV                0                          0                0
XLY                0                          0                0

## リーク防止チェック

- `lookforward=False` を特徴量候補、`lookforward=True` をラベルとして時点整合を確認する。
- 特徴量側の `t1` は常に `t` 以下、ラベル側の `t1` は常に `t` 以上であることを検証する。


In [10]:
leak_checks = []
for ticker in TICKERS:
    s = adj[ticker].copy()

    feat_df = trend_scanning_labels_numba(
        s,
        span=SPAN,
        lookforward=False,
        use_log=USE_LOG,
        volatility_threshold=VOL_THRESHOLD,
    )
    lbl_df = trend_scanning_labels_numba(
        s,
        span=SPAN,
        lookforward=True,
        use_log=USE_LOG,
        volatility_threshold=VOL_THRESHOLD,
    )

    check_temporal_integrity(feat_df, lookforward=False)
    check_temporal_integrity(lbl_df, lookforward=True)

    common_idx = feat_df.index.intersection(lbl_df.index)
    merged = pd.DataFrame(index=common_idx)
    merged['feat_t1'] = feat_df.loc[common_idx, 't1']
    merged['label_t1'] = lbl_df.loc[common_idx, 't1']

    cond_feat = bool((merged['feat_t1'] <= merged.index).all())
    cond_lbl = bool((merged['label_t1'] >= merged.index).all())

    # t+1 ターゲット整合の簡易検証
    fwd_ret_1 = s.shift(-1) / s - 1.0
    aligned_target = fwd_ret_1.reindex(common_idx)
    cond_target = bool(aligned_target.index.equals(common_idx))

    leak_checks.append({
        'ticker': ticker,
        'feature_past_only': int(cond_feat),
        'label_future_only': int(cond_lbl),
        't_plus_1_target_aligned': int(cond_target),
        'n_common_rows': int(len(common_idx)),
    })

leak_check_df = pd.DataFrame(leak_checks).set_index('ticker').sort_index()
leak_check_df


feature_past_only  label_future_only  t_plus_1_target_aligned  \
ticker                                                                  
XLB                     1                  1                        1   
XLE                     1                  1                        1   
XLF                     1                  1                        1   
XLI                     1                  1                        1   
XLK                     1                  1                        1   
XLP                     1                  1                        1   
XLU                     1                  1                        1   
XLV                     1                  1                        1   
XLY                     1                  1                        1   

        n_common_rows  
ticker                 
XLB              3609  
XLE              3609  
XLF              3609  
XLI              3609  
XLK              3609  
XLP              3609  
XLU              3609  
XLV              3609  
XLY              3609

In [11]:
acceptance = {}
acceptance['bin_match_all_1.0'] = bool((all_metrics_df['bin_match_rate'] == 1.0).all())
acceptance['window_match_ge_0.999'] = bool((all_metrics_df['window_match_rate'] >= 0.999).all())
acceptance['t_value_mae_le_1e-4'] = bool((all_metrics_df['t_value_mae'] <= 1e-4).all())
acceptance['t_value_max_abs_diff_le_1e-2'] = bool((all_metrics_df['t_value_max_abs_diff'] <= 1e-2).all())
acceptance['no_non_finite'] = bool((dist_flags['non_finite_flag'] == 0).all())
acceptance['leak_checks_all_pass'] = bool((leak_check_df[['feature_past_only', 'label_future_only', 't_plus_1_target_aligned']] == 1).all().all())

acceptance_df = pd.DataFrame.from_dict(acceptance, orient='index', columns=['passed'])
acceptance_df


passed
bin_match_all_1.0               True
window_match_ge_0.999           True
t_value_mae_le_1e-4             True
t_value_max_abs_diff_le_1e-2    True
no_non_finite                   True
leak_checks_all_pass            True

In [12]:
run_ts = datetime.now().astimezone().strftime('%Y-%m-%d %H:%M:%S %z')

versions = {
    'python': sys.version.split()[0],
    'numpy': np.__version__,
    'pandas': pd.__version__,
    'statsmodels': sm.__version__,
}
try:
    import numba
    versions['numba'] = numba.__version__
except Exception:
    versions['numba'] = 'N/A'

summary_cols = [
    'n_rows', 'bin_match_rate', 'window_match_rate',
    't_value_mae', 't_value_max_abs_diff',
    'reference_time_sec', 'numba_time_sec', 'speedup_x'
]
summary_table = all_metrics_df[summary_cols].copy()

lines = []
lines.append('# Phase B Evidence Log (minimal)')
lines.append('')
lines.append('## 1. Run metadata')
lines.append(f'- Run timestamp: {run_ts}')
lines.append(f'- Notebook: `{NOTEBOOK_PATH}`')
lines.append(f'- Data: `{DATA_PATH}`')
lines.append('')
lines.append('## 2. Dependency versions')
for k, v in versions.items():
    lines.append(f'- {k}: {v}')
lines.append('')
lines.append('## 3. Parameters')
lines.append(f'- span: {SPAN}')
lines.append(f'- volatility_threshold: {VOL_THRESHOLD}')
lines.append(f'- use_log: {USE_LOG}')
lines.append(f'- tickers: {TICKERS}')
lines.append('')
lines.append('## 4. Consistency summary (reference vs numba)')
lines.append(summary_table.to_markdown(floatfmt='.6g'))
lines.append('')
lines.append('## 5. Acceptance checks')
for key, row in acceptance_df.iterrows():
    lines.append(f'- {key}: {bool(row["passed"])}')
lines.append('')
lines.append('')
lines.append('## 6. Full-SMFit check (XLK subset, span=(5,20), n=700)')
if 'smfit_compare_df' in globals():
    lines.append(smfit_compare_df[['bin_match_rate', 'window_match_rate', 't_value_mae', 't_value_max_abs_diff']].to_markdown(floatfmt='.6g'))
else:
    lines.append('- smfit_compare_df not available')

lines.append('')
lines.append('## 7. Generated artifacts')
lines.append(f'- `{NOTEBOOK_PATH}`')
lines.append(f'- `{OUTPUT_EVIDENCE_PATH}`')

OUTPUT_EVIDENCE_PATH.write_text("\n".join(lines) + "\n", encoding='utf-8')
print('Evidence written to:', OUTPUT_EVIDENCE_PATH)
print("\n".join(lines[:18]))


Evidence written to: /Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/outputs/20260223/phase_b_evidence.md
# Phase B Evidence Log (minimal)

## 1. Run metadata
- Run timestamp: 2026-02-23 17:47:45 +0900
- Notebook: `/Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/trend_scanning_reproduction.ipynb`
- Data: `/Users/kencharoff/workspace/projects/momentum/p03_trend-scanning/data/prices.csv`

## 2. Dependency versions
- python: 3.12.12
- numpy: 2.4.2
- pandas: 3.0.0
- statsmodels: 0.14.6
- numba: 0.64.0

## 3. Parameters
- span: (5, 100)
- volatility_threshold: 0.05
- use_log: True
